In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_google_genai langchain_community langchain_core tavily-python wikipedia

In [ ]:
form langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")